## Library

In [21]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from tqdm import tqdm
from skimage.transform import resize
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Init GPU

In [22]:
# Initialize GPU Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

print(device)

GPU: Tesla P100-PCIE-16GB is available.
cuda


In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Config Info

In [24]:
# Constants
HEIGHT = 224
WIDTH = 224
CHANNELS = 3

TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
TEST_BATCH_SIZE = 4
TEST_SIZE = 0.15
VALID_SIZE = 0.15

MAX_SLICES = 60
SHAPE = (HEIGHT, WIDTH, CHANNELS)

NUM_EPOCHS = 50


In [25]:
# Kaggle and local switch
import os

KAGGLE = os.path.exists('/kaggle')
print("Running on Kaggle" if KAGGLE else "Running locally")

DATA_DIR = '/kaggle/input/rsna-mil-training/' if KAGGLE else '../rsna-mil-training/'
DICOM_DIR = DATA_DIR + 'rsna-mil-training'
CSV_PATH = DATA_DIR + 'training_1000_scan_subset.csv' if KAGGLE else './data_analyze/training_1000_scan_subset.csv'

dicom_dir = DICOM_DIR if KAGGLE else DATA_DIR
# Load patient scan labels
patient_scan_labels = pd.read_csv(CSV_PATH)

Running on Kaggle


## Data Preprocessing

In [26]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    
    # Resize
    img = cv2.resize(img, SHAPE[:2], interpolation = cv2.INTER_LINEAR)
   
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    
    bsb_img = np.stack([brain_img, subdural_img, soft_img], axis=-1)
    return bsb_img.astype(np.float16)

In [27]:
def preprocess_slice(slice, target_size=(HEIGHT, WIDTH)):
    # Check if type of slice is dicom 
    if (type(slice) == np.ndarray):
        slice = resize(slice, target_size, anti_aliasing=True)
        brain_window = apply_windowing(slice, window=(40, 80))
        subdural_window = apply_windowing(slice, window=(80, 200))
        bone_window = apply_windowing(slice, window=(600, 2800))
        
        multichannel_slice = np.stack([brain_window, subdural_window, bone_window], axis=-1)
        return multichannel_slice.astype(np.float16)
    else:
        slice = bsb_window(slice)
        return slice.astype(np.float16)

def apply_windowing(slice, window):
    window_width, window_level = window
    lower_bound = window_level - window_width // 2
    upper_bound = window_level + window_width // 2
    
    windowed_slice = np.clip(slice, lower_bound, upper_bound)
    windowed_slice = (windowed_slice - lower_bound) / (upper_bound - lower_bound)
    return windowed_slice

In [28]:
def read_dicom_folder(folder_path):
    slices = []
    for filename in sorted(os.listdir(folder_path))[:MAX_SLICES]:  # Limit to MAX_SLICES
        if filename.endswith(".dcm"):
            file_path = os.path.join(folder_path, filename)
            ds = pydicom.dcmread(file_path)
            slices.append(ds)
            
    # Sort slices by images position (z-coordinate) in ascending order
    slices = sorted(slices, key=lambda x: float(x.ImagePositionPatient[2]))
    
    # Pad with black images if necessary
    while len(slices) < MAX_SLICES:
        slices.append(np.zeros_like(slices[0].pixel_array))
    
    return slices[:MAX_SLICES]  # Ensure we return exactly MAX_SLICES

## Split Dataset

In [29]:
def split_dataset(patient_scan_labels, test_size=TEST_SIZE, val_size=VALID_SIZE, random_state=42):
    """
    Split the dataset into training, validation, and testing sets while maintaining the same ratio of labels.

    Args:
        patient_scan_labels (pd.DataFrame): The DataFrame containing patient scan labels.
        test_size (float): The proportion of the dataset to include in the test split.
        val_size (float): The proportion of the training set to include in the validation split.
        random_state (int): The seed used by the random number generator.

    Returns:
        Tuple: train_labels, val_labels, test_labels
    """
    # If any of the hemorrhage indicators is 1, the label is 1, otherwise 0
    patient_scan_labels['label'] = patient_scan_labels[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].any(axis=1).astype(int)

    # Extract the labels from the DataFrame
    labels = patient_scan_labels['label']

    # First, split off the test set
    train_val_labels, test_labels = train_test_split(
        patient_scan_labels, 
        test_size=test_size, 
        stratify=labels, 
        random_state=random_state
    )

    # Calculate the validation size relative to the train_val set
    val_size_adjusted = val_size / (1 - test_size)

    # Split the train_val set into train and validation sets
    train_labels, val_labels = train_test_split(
        train_val_labels, 
        test_size=val_size_adjusted, 
        stratify=train_val_labels['label'], 
        random_state=random_state
    )

    return train_labels, val_labels, test_labels

In [30]:
def process_patient_data(dicom_dir, row):
    patient_id = row['patient_id'].replace('ID_', '')
    study_instance_uid = row['study_instance_uid'].replace('ID_', '')
    
    folder_name = f"{patient_id}_{study_instance_uid}"
    folder_path = os.path.join(dicom_dir, folder_name)
    
    if os.path.exists(folder_path):
        slices = read_dicom_folder(folder_path)
        
        # Vì đã có dòng stack rồi nên có thể dòng này không cần thiết
        preprocessed_slices = [preprocess_slice(slice) for slice in slices]
        
        # Thêm chiều depth
        preprocessed_slices = np.stack(preprocessed_slices, axis=0)  # (depth, height, width, channels)
        preprocessed_slices = np.transpose(preprocessed_slices, (3, 0, 1, 2))  # (channels, depth, height, width)
        
        label = 1 if row[['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']].any() else 0
        
        return preprocessed_slices, label
    else:
        print(f"Folder not found: {folder_path}")
        return None, None

In [31]:
class TrainDatasetGenerator(Dataset):
    """
    A custom dataset class for training data.
    """
    def __init__(self, data_dir, patient_scan_labels):
        self.data_dir = data_dir
        self.patient_scan_labels = patient_scan_labels

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.data_dir, row)
        
        if preprocessed_slices is not None:
            # Convert the list of numpy arrays to a single numpy array
            preprocessed_slices = np.array(preprocessed_slices)  # Convert to numpy array
            return torch.tensor(preprocessed_slices, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        else:
            return None, None  # Handle the case where the folder is not found

class TestDatasetGenerator(Dataset):
    """
    A custom dataset class for testing data.
    """
    def __init__(self, data_dir, patient_scan_labels):
        self.data_dir = data_dir
        self.patient_scan_labels = patient_scan_labels

    def __len__(self):
        return len(self.patient_scan_labels)

    def __getitem__(self, idx):
        row = self.patient_scan_labels.iloc[idx]
        preprocessed_slices, label = process_patient_data(self.data_dir, row)
        
        if preprocessed_slices is not None:
            # Convert the list of numpy arrays to a single numpy array
            preprocessed_slices = np.array(preprocessed_slices)  # Convert to numpy array
            return torch.tensor(preprocessed_slices, dtype=torch.float32), torch.tensor(label, dtype=torch.long)
        else:
            return None, None  # Handle the case where the folder is not found

In [32]:
def get_train_loader(dicom_dir, patient_scan_labels, batch_size=TRAIN_BATCH_SIZE, shuffle=True):
    train_dataset = TrainDatasetGenerator(dicom_dir, patient_scan_labels)
    return DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4, pin_memory=True)

def get_test_loader(dicom_dir, patient_scan_labels, batch_size=TEST_BATCH_SIZE):
    test_dataset = TestDatasetGenerator(dicom_dir, patient_scan_labels)
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

## CNN Feature Extractor

In [33]:
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.fc1 = nn.Linear(in_channels, in_channels // reduction, bias=False).to(device)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels, bias=False).to(device)

    def forward(self, x):
        batch_size, channels, _, _, _ = x.size()
    
        avg_pool = F.adaptive_avg_pool3d(x, 1).view(batch_size, channels)
        max_pool = F.adaptive_max_pool3d(x, 1).view(batch_size, channels)

        avg_out = self.fc2(F.relu(self.fc1(avg_pool)))
        max_out = self.fc2(F.relu(self.fc1(max_pool)))

        out = torch.sigmoid(avg_out + max_out).view(batch_size, channels, 1, 1, 1)
        return out * x

In [34]:
class ResidualBlock3D(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock3D, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        
        # Apply channel attention methods
        out = ChannelAttention(out.size(1))(out)
        
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [35]:
# class ResNet3D_MIL(nn.Module):
#     def __init__(self, block, num_blocks, num_classes=2):
#         super(ResNet3D_MIL, self).__init__()
#         self.in_channels = 64
# 
#         self.conv1 = nn.Conv3d(3, 64, kernel_size=7, stride=(1, 2, 2), padding=3, bias=False)
#         self.bn1 = nn.BatchNorm3d(64)
#         self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
# 
#         self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
#         self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
#         self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
#         self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
# 
#         self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
#         self.fc = nn.Linear(512, num_classes)
#         
#         self.dropout = nn.Dropout(0.25)
# 
#     def _make_layer(self, block, out_channels, num_blocks, stride):
#         strides = [stride] + [1] * (num_blocks - 1)
#         layers = []
#         for stride in strides:
#             layers.append(block(self.in_channels, out_channels, stride))
#             self.in_channels = out_channels
#         return nn.Sequential(*layers)
# 
#     def forward(self, x):
#         # x shape: (batch_size, num_instances, channels, depth, height, width)
#         batch_size, c, d, h, w = x.size()
#         x = x.view(batch_size, c, d, h, w)
# 
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = self.maxpool(out)
# 
#         out = self.layer1(out)
#         out = self.dropout(out)
#         out = self.layer2(out)
#         out = self.dropout(out)
#         out = self.layer3(out)
#         out = self.dropout(out)
#         out = self.layer4(out)
# 
#         out = self.avgpool(out)
#         out = out.view(out.size(0), -1)
#         out = self.dropout(out)
#         out = self.fc(out)
# 
#         # MIL aggregation (max pooling over instances)
#         out = torch.max(out, dim=1)[0]
# 
#         return out
# 
# def ResNet3D18_MIL():
#     return ResNet3D_MIL(ResidualBlock3D, [2, 2, 2, 2])

In [36]:
class ResNet3D_MIL(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2):
        super(ResNet3D_MIL, self).__init__()
        self.in_channels = 16

        self.conv1 = nn.Conv3d(3, 16, kernel_size=7, stride=(1, 2, 2), padding=3, bias=False)
        self.bn1 = nn.BatchNorm3d(16)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 32, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 32, num_blocks[3], stride=2)
        self.layer5 = self._make_layer(block, 32, num_blocks[4], stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(32, num_classes)
        
        self.dropout = nn.Dropout(0.25)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        # x shape: (batch_size, num_instances, channels, depth, height, width)
        batch_size, c, d, h, w = x.size()
        x = x.view(batch_size, c, d, h, w)

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.maxpool(out)
        out = self.layer2(out)
        out = self.maxpool(out)
        out = self.layer3(out)
        out = self.maxpool(out)
        out = self.layer4(out)
        out = self.maxpool(out)
        out = self.layer5(out)

        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        out = self.fc(out)

        # MIL aggregation (max pooling over instances)
        out = torch.max(out, dim=1)[0]

        return out

def ResNet3D18_MIL():
    return ResNet3D_MIL(ResidualBlock3D, [2, 2, 2, 2, 2])

In [37]:
def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001, device='cuda'):
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    best_val_accuracy = 0.0
    best_model = None

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_predictions = []
        train_labels = []

        for batch_data, batch_labels in train_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.float().to(device)

            optimizer.zero_grad()
            outputs = model(batch_data).squeeze()
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_predictions.extend((outputs > 0).cpu().numpy())
            train_labels.extend(batch_labels.cpu().numpy())

        train_accuracy = accuracy_score(train_labels, train_predictions)
        train_precision = precision_score(train_labels, train_predictions)
        train_recall = recall_score(train_labels, train_predictions)
        train_f1 = f1_score(train_labels, train_predictions)

        # Validation
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_labels = []

        with torch.no_grad():
            for batch_data, batch_labels in val_loader:
                batch_data = batch_data.to(device)
                batch_labels = batch_labels.float().to(device)

                outputs = model(batch_data).squeeze()
                loss = criterion(outputs, batch_labels)

                val_loss += loss.item()
                val_predictions.extend((outputs > 0).cpu().numpy())
                val_labels.extend(batch_labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_predictions)
        val_precision = precision_score(val_labels, val_predictions)
        val_recall = recall_score(val_labels, val_predictions)
        val_f1 = f1_score(val_labels, val_predictions)

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()

    # Load best model
    model.load_state_dict(best_model)
    return model

In [38]:
def test_model(model, test_loader, device='cuda'):
    model = model.to(device)
    model.eval()
    test_predictions = []
    test_labels = []

    with torch.no_grad():
        for batch_data, batch_labels in test_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.float().to(device)

            outputs = model(batch_data).squeeze()
            test_predictions.extend((outputs > 0).cpu().numpy())
            test_labels.extend(batch_labels.cpu().numpy())

    test_accuracy = accuracy_score(test_labels, test_predictions)
    test_precision = precision_score(test_labels, test_predictions)
    test_recall = recall_score(test_labels, test_predictions)
    test_f1 = f1_score(test_labels, test_predictions)

    print(f"Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")

    return test_accuracy, test_precision, test_recall, test_f1

In [39]:
def plot_roc_curve(model, data_loader, device):
    model.eval()
    all_labels = []
    all_scores = []
    with torch.no_grad():
        for batch_data, batch_labels in data_loader:
            batch_data = batch_data.to(device)
            outputs = model(batch_data).squeeze()
            all_scores.extend(outputs.cpu().numpy())
            all_labels.extend(batch_labels.numpy())
    
    fpr, tpr, _ = roc_curve(all_labels, all_scores)
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

In [40]:
if __name__ == "__main__":
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = ResNet3D18_MIL()
    
    train_labels, val_labels, test_labels = split_dataset(patient_scan_labels, test_size=TEST_SIZE)
    
    train_loader = get_train_loader(dicom_dir, train_labels, batch_size=4)
    val_loader = get_train_loader(dicom_dir, val_labels, batch_size=2)
    test_loader = get_test_loader(dicom_dir, test_labels, batch_size=2)
    
    trained_model = train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001)
    
    # Save the model
    torch.save(trained_model.state_dict(), 'trained_model.pth')
    
    plot_roc_curve(trained_model, test_loader, device)

Epoch 1/50
Train Loss: 123.5617, Accuracy: 0.5329, Precision: 0.4452, Recall: 0.4407, F1: 0.4429
Val Loss: 51.4943, Accuracy: 0.5133, Precision: 0.4000, Recall: 0.3175, F1: 0.3540
Epoch 2/50
Train Loss: 123.1375, Accuracy: 0.5543, Precision: 0.4647, Recall: 0.3797, F1: 0.4179
Val Loss: 50.8636, Accuracy: 0.5533, Precision: 0.4167, Recall: 0.1587, F1: 0.2299
Epoch 3/50
Train Loss: 119.4613, Accuracy: 0.5700, Precision: 0.4766, Recall: 0.2068, F1: 0.2884
Val Loss: 51.4788, Accuracy: 0.6200, Precision: 0.5833, Recall: 0.3333, F1: 0.4242
Epoch 4/50
Train Loss: 117.8398, Accuracy: 0.5857, Precision: 0.5153, Recall: 0.2847, F1: 0.3668
Val Loss: 45.0702, Accuracy: 0.7467, Precision: 0.7193, Recall: 0.6508, F1: 0.6833
Epoch 5/50
Train Loss: 117.0957, Accuracy: 0.5971, Precision: 0.5330, Recall: 0.3559, F1: 0.4268
Val Loss: 46.9305, Accuracy: 0.6200, Precision: 0.6250, Recall: 0.2381, F1: 0.3448
Epoch 6/50
Train Loss: 115.2411, Accuracy: 0.5914, Precision: 0.5195, Recall: 0.4068, F1: 0.4563
Val

KeyboardInterrupt: 

In [ ]:
# Load the model and test it on test loader and print the results of classification to a csv 

# Load the model
model = ResNet3D18_MIL()
# Load the trained model
model.load_state_dict(torch.load('trained_model.pth'))
model.eval()

test_model(model, test_loader)

# Save the results to a csv file
results = []
with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.float().to(device)

        outputs = model(batch_data).squeeze()
        predictions = (outputs > 0).cpu().numpy()

        for i in range(len(predictions)):
            results.append({
                'prediction': predictions[i],
                'label': batch_labels[i].cpu().numpy()
            })

results_df = pd.DataFrame(results)
results_df.to_csv('results.csv', index=False)

results_df.head()

In [ ]:
import matplotlib.pyplot as plt

def plot_first_item(dataloader):
    first_item = next(iter(dataloader))
    
    images, labels = first_item[0]
    
    # Reshape images 
    images = images.squeeze(0)  
    images = images.permute(1, 0, 2, 3)  
    
    num_images = images.size(0)  
    
    images = images.numpy() 
    
    plt.figure(figsize=(15, 15))
    
    for i in range(num_images):
        plt.subplot(8, 8, i + 1) 
        plt.imshow(images[i][0], cmap='gray') 
        plt.title(f'Image {i + 1}')
        plt.axis('off')
    
    plt.tight_layout() 
    plt.show()

# Gọi hàm với test_loader
plot_first_item(test_loader)